In [73]:
import torch
from torch.autograd import Variable
from torch import optim, nn
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from argparse import Namespace
import pickle
import numpy as np
import time

In [19]:
NUM_CLASSES = 43
IMG_SIZE = 32

In [121]:
params = Namespace()
params.data = './data'
params.lr = 0.0001
params.batch_size = 64
params.seed = 7
params.cnn = '100, 150, 250, 350'
params.locnet = '200,300,200'
params.locnet2 = None
params.locnet3 = '150,150,150'
params.st = True
params.resume = False
params.epochs = 100
params.patience = 30
params.dropout = 0.5
params.use_pickle = True
params.train_pickle = '/scratch/as10656/traffic/train_extended_preprocessed.p'
params.extra_debug = True

In [8]:
# torch.cuda.manual_seed(params.seed)
torch.manual_seed(params.seed);

In [116]:
class TrafficSignsDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.from_numpy(images)
        self.images = self.images.permute(0, 3, 1, 2)
        self.labels = torch.LongTensor(labels.argmax(1))
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        return self.images[index], self.labels[index]

In [82]:
from sklearn.model_selection import train_test_split
class Utils:
    def __init__(self):
        self.train_data_transforms = transforms.Compose([
            transforms.Resize(32, 32)
        ])
        self.val_data_transforms = transforms.Compose([
            transforms.Resize(32, 32)
        ])
    def load_pickled_data(self, file, columns):
        with open(file, mode='rb') as f:
            dataset = pickle.load(f)
        return tuple(map(lambda c: dataset[c], columns))
    
    def get_dataset(self, params):
        if params.use_pickle:
            data_images, data_labels = self.load_pickled_data(params.train_pickle, ['features', 'labels'])
            train_images, val_images, train_labels, val_labels = train_test_split(data_images, 
                                                                                  data_labels, 
                                                                                  test_size=0.25) 
            return TrafficSignsDataset(train_images, train_labels), TrafficSignsDataset(val_images, val_labels)
        else:
            train_dataset = datasets.ImageFolder(params.data + '/train_images',
                                                 transform=self.train_data_transforms)
            val_dataset = datasets.ImageFolder(self.params.data + '/val_images',
                                               transform=self.val_data_transforms)
            return train_dataset, val_dataset
    def get_convnet_output_size(self, network, input_size=IMG_SIZE):
        input_size = input_size or IMG_SIZE

        if type(network) != list:
            network = [network]

        in_channels = network[0].conv.in_channels

        output = Variable(torch.ones(1, in_channels, input_size, input_size))
        output.require_grad = False
        for conv in network:
            output = conv.forward(output)

        return np.asscalar(np.prod(output.data.shape)), output.data.size()[2]
    def get_time_hhmmss(self, start = None):
        """
        Calculates time since `start` and formats as a string.
        """
        if start is None:
            return time.strftime("%Y/%m/%d %H:%M:%S")
        end = time.time()
        m, s = divmod(end - start, 60)
        h, m = divmod(m, 60)
        time_str = "%02d:%02d:%02d" % (h, m, s)
        return time_str  


utils = Utils()

In [11]:
class ConvNet(nn.Module):

    def __init__(self, in_c, out_c,
                 kernel_size,
                 padding_size='same',
                 pool_stride=2,
                 batch_norm=True):
        super().__init__()

        if padding_size == 'same':
            padding_size = kernel_size // 2
        self.conv = nn.Conv2d(in_c, out_c, kernel_size, padding=padding_size)
        self.max_pool2d = nn.MaxPool2d(pool_stride, stride=pool_stride)
        self.batch_norm = batch_norm
        self.batch_norm_2d = nn.BatchNorm2d(out_c)

    def forward(self, x):
        x = self.max_pool2d(nn.functional.leaky_relu(self.conv(x)))

        if self.batch_norm:
            return self.batch_norm_2d(x)
        else:
            return x

In [12]:
class Classifier(nn.Module):
    def __init__(self, input_nbr, out_nbr):
        super(Classifier, self).__init__()
        self.input_nbr = input_nbr
        self.lin = nn.Linear(input_nbr, out_nbr)

    def forward(self, x):
        return self.lin(x)

In [13]:
class SoftMaxClassifier(Classifier):
    def __init__(self, in_len, out_len):
        super().__init__(in_len, out_len)

    def forward(self, x):
        x = super().forward(x)
        return nn.functional.log_softmax(x)


In [14]:
class FullyConnected(nn.Module):
    def __init__(self, input_nbr, out_nbr):
        super(FullyConnected, self).__init__()
        self.input_nbr = input_nbr
        self.lin = nn.Linear(input_nbr, out_nbr)
        self.rel = nn.LeakyReLU()
        self.dropout = nn.Dropout()

    def forward(self, input):
        return self.dropout(self.rel(self.lin(input)))

In [39]:
class LocalizationNetwork(nn.Module):
    nbr_params = 6
    init_bias = torch.Tensor([1, 0, 0, 0, 1, 0])

    def __init__(self, conv_params, kernel_sizes,
                 input_size, input_channels=1):
        super(LocalizationNetwork, self).__init__()

        if not kernel_sizes:
            kernel_sizes = [5, 5]

        if len(kernel_sizes) != 2:
            raise Exception("Number of kernel sizes != 2")

        self.conv1 = ConvNet(input_channels, conv_params[0],
                             kernel_size=kernel_sizes[0],
                             batch_norm=False)
        self.conv2 = ConvNet(conv_params[0], conv_params[1],
                             kernel_size=kernel_sizes[1],
                             batch_norm=False)
        conv_output_size, _ = utils.get_convnet_output_size([self.conv1, self.conv2],
                                                            input_size)

        self.fc = FullyConnected(conv_output_size, conv_params[2])
        self.classifier = Classifier(conv_params[2], self.nbr_params)

        self.classifier.lin.weight.data.fill_(0)
        self.classifier.lin.bias.data = torch.FloatTensor([1, 0, 0, 0, 1, 0])
        self.dropout = nn.Dropout2d()

    def forward(self, x):
        x = self.dropout(self.conv1(x))
        conv_output = self.dropout(self.conv2(x))
        conv_output = conv_output.view(conv_output.size()[0], -1)
        return self.classifier(self.fc(conv_output))

In [40]:
class SpatialTransformerNetwork(nn.Module):
    def __init__(self, params, kernel_sizes, input_size=IMG_SIZE,
                 input_channels=1):
        super(SpatialTransformerNetwork, self).__init__()
        self.localization_network = LocalizationNetwork(params,
                                                        kernel_sizes,
                                                        input_size,
                                                        input_channels)

    def forward(self, input):
        out = self.localization_network(input)
        out = out.view(out.size()[0], 2, 3)
        grid = nn.functional.affine_grid(out, input.size())
        return nn.functional.grid_sample(input, grid)


In [65]:
class GeneralNetwork(nn.Module):
    def __init__(self, opt):
        super(GeneralNetwork, self).__init__()

        if not opt.cnn:
            opt.cnn = '100, 150, 250, 350'
        kernel_sizes = [7, 4, 4]
        conv_params = list(map(int, opt.cnn.split(",")))

        self.conv1 = ConvNet(1, conv_params[0], kernel_size=kernel_sizes[0],
                             padding_size=0)
        self.conv2 = ConvNet(conv_params[0], conv_params[1],
                             kernel_size=kernel_sizes[1],
                             padding_size=0)

        conv_output_size, _ = utils.get_convnet_output_size([self.conv1, self.conv2])

        self.fc = FullyConnected(conv_output_size, conv_params[2])
        self.classifier = SoftMaxClassifier(conv_params[2], NUM_CLASSES)

        self.locnet_1 = None
        if opt.st and opt.locnet:
            params = list(map(int, opt.locnet.split(",")))
            self.locnet_1 = SpatialTransformerNetwork(params,
                                                      kernel_sizes=[7, 5])

        self.locnet_2 = None
        if opt.st and opt.locnet2:
            params = list(map(int, opt.locnet2.split(",")))
            _, current_size = utils.get_convnet_output_size([self.conv1])
            self.locnet_2 = SpatialTransformerNetwork(params,
                                                      [5, 3],
                                                      current_size,
                                                      conv_params[0])
        self.dropout = nn.Dropout2d()

    def forward(self, x):
        if self.locnet_1:
            x = self.locnet_1(x)

        x = self.conv1(x)

        if self.locnet_2:
            x = self.locnet_2(x)

        return self.classifier(self.fc(self.dropout(self.conv2(x))))

In [79]:
class IDSIANetwork(GeneralNetwork):
    def __init__(self, opt):
        super().__init__(opt)
        conv_params = list(map(int, opt.cnn.split(",")))

        self.conv3 = ConvNet(conv_params[1], conv_params[2], kernel_size=4,
                             padding_size=0)
        conv_output_size, _ = utils.get_convnet_output_size([self.conv1,
                                                      self.conv2,
                                                      self.conv3])
        self.fc = FullyConnected(conv_output_size, conv_params[3])
        self.classifier = SoftMaxClassifier(conv_params[3], NUM_CLASSES)

        self.locnet_3 = None
        if opt.st and opt.locnet3:
            params = list(map(int, opt.locnet3.split(",")))
            _, current_size = utils.get_convnet_output_size([self.conv1, self.conv2])
            self.locnet_3 = SpatialTransformerNetwork(params,
                                                      [3, 3],
                                                      current_size,
                                                      conv_params[1])

    def forward(self, x):
        if self.locnet_1:
            x = self.locnet_1(x)

        x = self.conv1(x)
        x = self.dropout(x)

        if self.locnet_2:
            x = self.locnet_2(x)

        x = self.conv2(x)
        x = self.dropout(x)

        if self.locnet_3:
            x = self.locnet_3(x)

        x = self.conv3(x)
        x = self.dropout(x)

        x = x.view(x.size()[0], -1)
        return self.classifier(self.fc(x))

In [43]:
class EarlyStopping(object):
    def __init__(self, model, optimizer, patience = 100, minimize = True):
        self.minimize = minimize
        self.patience = patience
        self.model = model
        self.optimizer = optimizer
        self.best_monitored_value = np.inf if minimize else 0.
        self.best_monitored_acc = 0. if minimize else np.inf
        self.best_monitored_epoch = 0

        self.restore_path = None

    def __call__(self, value, acc, epoch, rest):
        if (self.minimize and value < self.best_monitored_value) or (not self.minimize and value > self.best_monitored_value):
            self.best_monitored_value = value
            self.best_monitored_epoch = epoch
            self.best_monitored_acc = acc
            state = {
                'epoch': epoch + 1,
                'state_dict': self.model.state_dict(),
                'best': value,
                'best_acc': acc,
                'optimizer': self.optimizer.state_dict(),
            }            
            
            state.update(rest)
            self.restore_path = utils.save_checkpoint(state, True, "/scratch/as10656/nli-models/early_stopping_checkpoint")
        elif self.best_monitored_epoch + self.patience < epoch:
            if self.restore_path != None:
                checkpoint = utils.load_checkpoint(self.restore_path)
                self.best_monitored_value = checkpoint['best']
                self.best_monitored_acc = checkpoint['best_acc']
                self.best_monitored_epoch = checkpoint['epoch']
                self.model.load_state_dict(checkpoint['state_dict'])
                self.optimizer.load_state_dict(checkpoint['optimizer'])
            else:
                print("ERROR: Failed to restore session")
            return True
        
        return False
    
    def init_from_checkpoint(self, checkpoint):
        self.best_monitored_value = checkpoint['best']
        self.best_monitored_acc = checkpoint['best_acc']
        self.best_monitored_epoch = 0
    
    def print_info(self):
        print("Best loss: {0}, Best Accuracy: {1}, at epoch {2}"
              .format(self.best_monitored_value,
                      self.best_monitored_acc,
                      self.best_monitored_epoch))

In [44]:
def plot_curve(axis, params, train_column, valid_column, linewidth = 2, train_linestyle = "b-", valid_linestyle = "g-"):
    """
    Plots a pair of validation and training curves on a single plot.
    """
    train_values = params[train_column]
    valid_values = params[valid_column]
    epochs = train_values.shape[0]
    x_axis = np.arange(epochs)
    axis.plot(x_axis[train_values > 0], train_values[train_values > 0], train_linestyle, linewidth=linewidth, label="train")
    axis.plot(x_axis[valid_values > 0], valid_values[valid_values > 0], valid_linestyle, linewidth=linewidth, label="valid")
    return epochs

# Plots history of learning curves for a specific model.
def plot_learning_curves(params):
    """
    Plots learning curves (loss and accuracy on both training and validation sets) for a model identified by a parameters struct.
    """
    curves_figure = plt.figure(figsize = (10, 4))
    axis = curves_figure.add_subplot(1, 2, 1)
    epochs_plotted = plot_curve(axis, params, train_column = "train_acc", valid_column = "val_acc")

    plt.grid()
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.ylim(50., 115.)
    plt.xlim(0, epochs_plotted)

    axis = curves_figure.add_subplot(1, 2, 2)
    epochs_plotted = plot_curve(axis, params, train_column = "train_loss", valid_column = "val_loss")

    plt.grid()
    plt.legend()
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.ylim(0.0001, 10.)
    plt.xlim(0, epochs_plotted)
    plt.yscale("log")

In [114]:
class Trainer:
    def __init__(self, params, train_data, val_data):
        self.params = params
        self.train_data = train_data
        self.val_data = val_data
        self.epochs = params.epochs
        print("Creating dataloaders")
        self.cuda_available = torch.cuda.is_available()
        
        self.train_loader = DataLoader(dataset=train_data,
                                       shuffle=True,
                                       batch_size=params.batch_size,
                                       pin_memory=self.cuda_available)
        self.val_loader = DataLoader(dataset=val_data,
                                     shuffle=False,
                                     batch_size=params.batch_size,
                                     pin_memory=self.cuda_available)
        
        self.string_fixer = "=========="

        
    def train(self):
        criterion = nn.CrossEntropyLoss()
        model = IDSIANetwork(self.params)
        optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                                      model.parameters()),
                               lr=params.lr)

        start_epoch = 0
        best_prec = 0
        self.start_time = time.time()
        self.histories = {
            "train_loss": np.empty(0, dtype=np.float32),
            "train_acc": np.empty(0, dtype=np.float32),
            "val_loss": np.empty(0, dtype=np.float32),
            "val_acc": np.empty(0, dtype=np.float32)
        }
        
        
        self.early_stopping = EarlyStopping(model, optimizer, patience=self.params.patience)
        if self.params.resume:
            checkpoint = utils.load_checkpoint(self.params.resume)
            if checkpoint is not None:
                model.load_state_dict(checkpoint['state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer'])
                self.histories.update(checkpoint)
                self.early_stopping.init_from_checkpoint(checkpoint)
                print("Loaded model, Best Loss: %.8f, Best Acc: %.2f" % (checkpoint['best'], checkpoint['best_acc']))

        is_best = False

        if self.cuda_available:
            model = model.cuda()
        
        self.model = model
        
        model.train()
        print("Starting training")
        self.print_info()
        for epoch in range(start_epoch, params.epochs):
            for i, (images, labels) in enumerate(self.train_loader):
                images_batch = Variable(images)
                labels_batch = Variable(labels)

                if self.cuda_available:
                    images_batch = images_batch.cuda()
                    labels_batch = labels_batch.cuda(async=True)

                optimizer.zero_grad()
                output = model(images_batch)
                loss = criterion(output, labels_batch.long())
                loss.backward()
                optimizer.step()
                
                if self.params.extra_debug and (i + 1) % (self.params.batch_size * 4) == 0:
                    print(('Epoch: [{0}/{1}], Step: [{2}/{3}], Loss: {4},')
                             .format(epoch + 1,
                                     self.params.epochs,
                                     i + 1,
                                     len(self.train_loader),
                                     loss.data[0]))
            
            train_acc, train_loss = self.validate_model(self.train_loader, model)
            val_acc, val_loss = self.validate_model(self.val_loader, model)
            
            self.histories['train_loss'] = np.append(self.histories['train_loss'], [train_loss])
            self.histories['val_loss'] = np.append(self.histories['val_loss'], [val_loss])
            self.histories['val_acc'] = np.append(self.histories['val_acc'], [val_acc])
            self.histories['train_acc'] = np.append(self.histories['train_acc'], [train_acc])
            
            if not self.early_stopping(val_loss, val_acc, epoch, self.histories):
                self.print_train_info(epoch, train_acc, train_loss, val_acc, val_loss)
            else:
                print("Early stopping activated")
                print("Restoring earlier state and stopping")
                self.early_stopping.print_info()
                plot_learning_curves(self.histories)
                plt.show()
                break
            

            
    def validate_model(self, loader, model):
        model.eval()
        correct = 0
        total = 0
        total_loss = 0

        for images, hypo, labels in loader:
            images_batch = Variable(images, volatile=True)
            labels_batch = Variable(labels.long())

            if self.cuda_available:
                images_batch = images_batch.cuda()
                labels_batch = labels_batch.cuda()

            output = model(images_batch, hypo_batch)
            loss = nn.functional.cross_entropy(output, labels_batch.long(), size_average=False)
            total_loss += len(images_batch) * loss.data
            total += len(labels_batch)
            
            if not self.cuda_available:
                correct += (labels_batch == output.max(1)[1]).data.cpu().numpy().sum()
            else:
                correct += (labels_batch == output.max(1)[1]).data.sum()
                
        model.train()

        average_loss = total_loss[0] / total
        return correct / total * 100, average_loss

    def print_info(self):
        print(self.string_fixer + " Data " + self.string_fixer)
        print("Training set: %d examples" % (len(self.train_data)))
        print("Validation set: %d examples" % (len(self.val_data)))
        print("Timestamp: %s" % utils.get_time_hhmmss())
        
        print(self.string_fixer + " Params " + self.string_fixer)
    
        print("Learning Rate: %f" % self.params.lr)
        print("Dropout (p): %f" % self.params.dropout)
        print("Batch Size: %d" % self.params.batch_size)
        print("Epochs: %d" % self.params.epochs)
        print("Patience: %d" % self.params.patience)
        print("Resume: %s" % self.params.resume)
        
    def print_train_info(self, epoch, train_acc, train_loss, val_acc, val_loss):
        print((self.string_fixer + " Epoch: {0}/{1} " + self.string_fixer)
              .format(epoch + 1, self.params.epochs))
        print("Train Loss: %.8f, Train Acc: %.2f" % (train_loss, train_acc))
        print("Validation Loss: %.8f, Validation Acc: %.2f" % (val_loss, val_acc))
        self.early_stopping.print_info()
        print("Elapsed Time: %s" % (utils.get_time_hhmmss(self.start_time)))
        print("Current timestamp: %s" % (utils.get_time_hhmmss()))

In [117]:
train_dataset, val_dataset = utils.get_dataset(params)

In [122]:
trainer = Trainer(params, train_dataset, val_dataset)

Creating dataloaders


In [ ]:
trainer.train()

Starting training
========== Data ==========
Training set: 521985 examples
Validation set: 173995 examples
Timestamp: 2017/11/20 16:46:08
========== Params ==========
Learning Rate: 0.000100
Dropout (p): 0.500000
Batch Size: 64
Epochs: 100
Patience: 30
Resume: False
Epoch: [1/100], Step: [256/8157], Loss: 3.5578806400299072,
Epoch: [1/100], Step: [512/8157], Loss: 3.469850778579712,
Epoch: [1/100], Step: [768/8157], Loss: 3.562333583831787,
Epoch: [1/100], Step: [1024/8157], Loss: 3.5242562294006348,
Epoch: [1/100], Step: [1280/8157], Loss: 3.643771171569824,
Epoch: [1/100], Step: [1536/8157], Loss: 3.514530897140503,
Epoch: [1/100], Step: [1792/8157], Loss: 3.372837781906128,
Epoch: [1/100], Step: [2048/8157], Loss: 3.5515120029449463,
Epoch: [1/100], Step: [2304/8157], Loss: 3.4567337036132812,
Epoch: [1/100], Step: [2560/8157], Loss: 3.5267200469970703,
Epoch: [1/100], Step: [2816/8157], Loss: 3.4223105907440186,
Epoch: [1/100], Step: [3072/8157], Loss: 3.483168601989746,
Epoch: [1/